In [17]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.svm import LinearSVC, OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import eli5
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
np.random.seed(500)
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/borisdejong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
df = pd.read_pickle('../data/v2.0-processed/df_premises.pickle')
df_test = pd.read_pickle('../data/v2.0-processed/df_premises_test.pickle')
df.head()
df_test.head()

,document,file_path,split,premise,type,logos,pathos,ethos
0,99,v2.0/positive/99.xml,positive,"For the sake of this thread, we'll exclude chi...",logos,True,False,False
1,99,v2.0/positive/99.xml,positive,"With a healthy child, income, neighborhood, pe...",logos,True,False,False
2,99,v2.0/positive/99.xml,positive,maybe a monetary incentive derived from public...,logos_pathos,True,True,False
3,99,v2.0/positive/99.xml,positive,"since it would only be a suspension of ""assist...",logos,True,False,False
4,99,v2.0/positive/99.xml,positive,"And poverty is a formidable obstacle but many,...",logos,True,False,False


In [19]:
# Lowercase
df['premise'] = df['premise'].str.lower()
df['logos'] = df['logos'].map({True: 'Logos', False: 'Not Logos'})
df['pathos'] = df['pathos'].map({True: 'Pathos', False: 'Not Pathos'})
df['ethos'] = df['ethos'].map({True: 'Ethos', False: 'Not Ethos'})
df['premise'] = [word_tokenize(entry) for entry in df['premise']]
df['text'] = [' '.join(entry) for entry in df['premise']]

df_test['premise'] = df_test['premise'].str.lower()
df_test['logos'] = df_test['logos'].map({True: 'Logos', False: 'Not Logos'})
df_test['pathos'] = df_test['pathos'].map({True: 'Pathos', False: 'Not Pathos'})
df_test['ethos'] = df_test['ethos'].map({True: 'Ethos', False: 'Not Ethos'})
df_test['premise'] = [word_tokenize(entry) for entry in df_test['premise']]
df_test['text'] = [' '.join(entry) for entry in df_test['premise']]

In [20]:
df['logos'].value_counts()
df_test['logos'].value_counts()

Logos        332
Not Logos    108
Name: logos, dtype: int64

In [21]:
# train, test = train_test_split(df, test_size=0.2, random_state=0, stratify=df['type'], shuffle=True)

In [22]:
# sem_type = 'pathos'
# train, test = train_test_split(df, test_size=0.2, random_state=0, stratify=df[sem_type])
# print(train[sem_type].value_counts())
# print(test[sem_type].value_counts())
# Train_X, Test_X, Train_Y, Test_Y = train['text'], test['text'], train[sem_type], test[sem_type]
# Encoder = LabelEncoder()
# Train_Y = Encoder.fit_transform(Train_Y)
# Test_Y = Encoder.fit_transform(Test_Y)
# print(np.count_nonzero(Train_Y == 1), np.count_nonzero(Train_Y == 0))
# print(np.count_nonzero(Test_Y == 1), np.count_nonzero(Test_Y == 0))
# Tfidf_vect = TfidfVectorizer(max_features=5000)
# Tfidf_vect.fit(df['text'])
# Train_X_Tfidf = Tfidf_vect.transform(Train_X)
# Test_X_Tfidf = Tfidf_vect.transform(Test_X)
# # Classifier - Algorithm - SVM
# # fit the training dataset on the classifier
# SVM = svm.LinearSVC(C=1.0)
# SVM.fit(Train_X_Tfidf, Train_Y) # predict the labels on validation dataset
# predictions_SVM = SVM.predict(Test_X_Tfidf) # Use accuracy_score function to get the accuracy
# # y = Encoder.inverse_transform(Test_Y)
# # pred = Encoder.inverse_transform(predictions_SVM)
# # print(f"SVM Accuracy Score for {sem_type} -> \n", classification_report(pred, y))
# print(f"SVM Accuracy Score for {sem_type} -> \n", classification_report(Test_Y, predictions_SVM))

In [23]:
train, dev = train_test_split(df, test_size=0.2, random_state=0, stratify=df['type'], shuffle=True)
test = df_test


print(test['pathos'].value_counts())
print(test['logos'].value_counts())
print(test['ethos'].value_counts())

def train_svm(sem_type):
    Train_X, Test_X, Dev_X, Train_Y, Test_Y, Dev_Y,  = train['text'], test['text'], dev['text'], train[sem_type], test[sem_type], dev[sem_type]

    # Construct pipeline
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('svc', LinearSVC()),
    ])

    pipe.fit(Train_X, Train_Y)

    predictions_SVM_dev = pipe.predict(Dev_X)
    predictions_SVM_test = pipe.predict(Test_X) # Use accuracy_score function to get the accuracy

    print(f"{sem_type} (dev set) -> \n", classification_report(Dev_Y, predictions_SVM_dev))

    print(f"{sem_type} (test set) -> \n", classification_report(Test_Y, predictions_SVM_test))

    eli5.show_weights(pipe)

Not Pathos    230
Pathos        210
Name: pathos, dtype: int64
Logos        332
Not Logos    108
Name: logos, dtype: int64
Not Ethos    387
Ethos         53
Name: ethos, dtype: int64


In [24]:
for sem_type in ['logos', 'ethos', 'pathos']:
    train_svm(sem_type=sem_type)

logos (dev set) -> 
               precision    recall  f1-score   support

       Logos       0.90      0.99      0.94       285
   Not Logos       0.77      0.24      0.36        42

    accuracy                           0.89       327
   macro avg       0.83      0.61      0.65       327
weighted avg       0.88      0.89      0.87       327

logos (test set) -> 
               precision    recall  f1-score   support

       Logos       0.83      1.00      0.91       332
   Not Logos       0.98      0.37      0.54       108

    accuracy                           0.84       440
   macro avg       0.90      0.68      0.72       440
weighted avg       0.87      0.84      0.82       440

ethos (dev set) -> 
               precision    recall  f1-score   support

       Ethos       0.67      0.11      0.19        18
   Not Ethos       0.95      1.00      0.97       309

    accuracy                           0.95       327
   macro avg       0.81      0.55      0.58       327
weighted a

/Users/borisdejong/PycharmProjects/ltp/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/borisdejong/PycharmProjects/ltp/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/borisdejong/PycharmProjects/ltp/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
